<a href="https://colab.research.google.com/github/lvllvl/SLAM/blob/main/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Section

## Run script: Connect to google drive

In [1]:
# Mount your drive, to save parts of your model, e.g., checkpoints, etc.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Run Script: Git clone repos

- comma10k repo
- project repo (SLAM)

### Run requirements.txt

In [3]:
# Clone a specific submodule
!git clone https://github.com/lvllvl/SLAM.git

# Download Comma10k repository
!git clone https://github.com/commaai/comma10k.git

# Install any dependencies
!pip install -r SLAM/requirements.txt


Cloning into 'SLAM'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 108 (delta 44), reused 86 (delta 27), pack-reused 0
Receiving objects: 100% (108/108), 21.02 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (44/44), done.
fatal: destination path 'comma10k' already exists and is not an empty directory.


## Setup folder structure

- Create new folders
- Move `comma10k` files into the right folder structure

In [4]:
# Create a directory for training and validation
!mkdir -p dataset_root/train/images
!mkdir -p dataset_root/train/masks
!mkdir -p dataset_root/val/images
!mkdir -p dataset_root/val/masks

# Move images for training folder
!mv comma10k/imgs/* dataset_root/train/images/
!mv comma10k/masks/* dataset_root/train/masks/

# Move images for validation folder
!mv comma10k/imgs2/* dataset_root/val/images/
!mv comma10k/masks2/* dataset_root/val/masks/

# Show all the folders so far
!ls
!ls dataset_root/

In [14]:
# Verify all same number of files are in the folders

!ls -l dataset_root/train/images | grep ^- | wc -l
!ls -l dataset_root/train/masks | grep ^- | wc -l

!ls -l dataset_root/val/images | grep ^- | wc -l
!ls -l dataset_root/val/masks | grep ^- | wc -l

9888
9888
2000
2000


In [20]:
!mv SLAM/*.py dataset_root/

# Model Section

## Segmentation Model: U-net Segmentaiton

In [24]:
!python dataset_root/train.py

Traceback (most recent call last):
  File "/content/dataset_root/train.py", line 4, in <module>
    from model import UNet
  File "/content/dataset_root/model.py", line 80, in <module>
    unet_model = UNet()
  File "/content/dataset_root/model.py", line 31, in __init__
    self.up_conv3 = self.up_conv( 1024, 512 )
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1695, in __getattr__
    raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")
AttributeError: 'UNet' object has no attribute 'up_conv'
